In [1]:
# !pip install import-ipynb

In [6]:
import pandas as pd
import numpy as np
import pyodbc
from scipy.stats import shapiro
from scipy.stats import kstest
from configparser import ConfigParser
from datetime import datetime
from scipy import stats as s
import statistics
# import import_ipynb
# import Entity_Risk

In [7]:
pd.set_option("display.max_column",200)
pd.set_option("display.float_format",lambda x:'%.5f' %x)

In [8]:
data_ind_U = pd.read_csv(r"/data/peer_updated.csv")

In [10]:
config = ConfigParser()
config.read(r'/notebooks/notebooks/config.ini')
connection_string = config.get('SERVER', 'connection')
conn = pyodbc.connect(connection_string)
cursor = conn.cursor()

In [11]:
risk_lkp = pd.read_sql_query("SELECT * FROM [FINCORE_DB].[Fincore_Analytics].[RISK_LKP_RULES] with (nolock) ", conn)
auto_high_rules =  risk_lkp[(risk_lkp['IS_ACTIVE_FLAG'] == 1) & (risk_lkp['RISK_TYPE'] == 'Auto High Rules ')]['RULE_NO'].to_list()
static_rules = risk_lkp[(risk_lkp['RISK_SCORE_STATIC_FLAG'] == 1) & (risk_lkp['IS_ACTIVE_FLAG'] == 1) & (risk_lkp['PERSON_TYPE'] == 'Individual')]['RULE_NO'].to_list()
non_stat_rules = risk_lkp[(risk_lkp['RISK_SCORE_STATIC_FLAG'] == 0) & (risk_lkp['IS_ACTIVE_FLAG'] == 1) & (risk_lkp['PERSON_TYPE'] == 'Individual')]['RULE_NO'].to_list()

stat_rules = tuple(static_rules)

In [12]:
et_risk_df_per = pd.read_sql_query("SELECT * FROM [Fincore_db].[Fincore_Analytics].[RISK_PERSON_SUMMARY_STG] with (nolock) ", conn)

In [14]:
non_stat_rules

['P001001000',
 'P001003000',
 'P002001001',
 'P002001010',
 'P002001011',
 'P002001012',
 'P002001015',
 'P002001016',
 'P002001018',
 'P002001019',
 'P002001002',
 'P002001021',
 'P002001023',
 'P002001024',
 'P002001025',
 'P002001026',
 'P002001027',
 'P002001028',
 'P002001029',
 'P002001030',
 'P002001031',
 'P002001032',
 'P002001033',
 'P002001034',
 'P002001004',
 'P002001005',
 'P002001007',
 'P002001008',
 'P002001009',
 'P002003001',
 'P002003002',
 'P002003003',
 'P002003005',
 'P002003006',
 'P002003007',
 'P002003008']

In [15]:
# Read risk_person_summary_stg_rules for the static rules
if len(non_stat_rules) == 1:
    et_risk_df_per_rules = pd.read_sql_query("SELECT * FROM [Fincore_db].[Fincore_Analytics].[RISK_PERSON_SUMMARY_STG_RULES] with (nolock) where RULE_NO in ('{}')".format(non_stat_rules[0]),conn)
    
elif(len(non_stat_rules)== 0):
    et_risk_df_per_rules = pd.read_sql_query("SELECT * FROM [Fincore_db].[Fincore_Analytics].[RISK_PERSON_SUMMARY_STG_RULES] with (nolock) where RULE_NO in ('')",conn)
    
else:
    non_stat_rules = tuple(non_stat_rules)
    et_risk_df_per_rules = pd.read_sql_query("SELECT * FROM [Fincore_db].[Fincore_Analytics].[RISK_PERSON_SUMMARY_STG_RULES] with (nolock) where RULE_NO in" + str (non_stat_rules) , conn)

# stat_rule_df = pd.read_sql_query("SELECT * FROM [FINCORE_DB].[Fincore_Analytics].[RISK_PERSON_SUMMARY_STG_RULES] where RULE_NO in "+str(tuple(static_rules)) , conn)


In [16]:
auto_high_rules

['P001001000', 'P001003000', 'R005001000']

In [18]:
auto_high_df = et_risk_df_per_rules[(et_risk_df_per_rules['RULE_NO'].isin(auto_high_rules)) & (et_risk_df_per_rules['RISK_SCORE_STATIC'] == 10)]

risk_per = pd.read_sql_query("SELECT * FROM [FINCORE_DB].[Fincore_Analytics].[RISK_PERSON]",conn)

auto_high_df = auto_high_df.groupby('PERSON_MASTER_ID')['RULE_NO'].unique().reset_index()

auto_high_df['RISK_RULE_LIST'] = auto_high_df['RULE_NO'].apply(lambda x:"_".join(x))

auto_high_df['RISK_SCORE'] = 10

In [19]:
auto_high_df

PERSON_MASTER_ID       RULE_NO RISK_RULE_LIST  RISK_SCORE
0                51  [P001003000]     P001003000          10
1               147  [P001003000]     P001003000          10

In [20]:
flaged_per_master_id = auto_high_df['PERSON_MASTER_ID'].to_list()
flaged_per_master_id

[51, 147]

In [39]:
config = ConfigParser()
config.read(r'/notebooks/notebooks/config.ini')
connection_string = config.get('SERVER', 'connection')
conn = pyodbc.connect(connection_string)
cursor = conn.cursor()

In [23]:
summary_stg_df = pd.read_sql_query("SELECT * FROM [FINCORE_DB].[Fincore_Analytics].[RISK_PERSON_SUMMARY_STG] with (nolock) ", conn)

In [24]:
summary_stg_df.head()

PERSON_MASTER_ID   PEER_GROUP_ID         PAN   DIN     CKYC_NUMBER  \
0                 1  Not Declared_0  JDJPK5455L  None  64500000000000   
1                 2  Not Declared_0  JDJPK5458L  None  65412564587515   
2                 3  Not Declared_0  JDJPK5459L  None  65412564587516   
3                 4  Not Declared_0  JDJPK5460L  None  65412564587517   
4                 5  Not Declared_0  JDJPK5461L  None  65412564587518   

                FULL_NAME FATHER_NAME MOTHER_NAME DATE_OF_BIRTH  GENDER  \
0  Satyawan Hira Deshmukh        Hira        None    1993-03-14    Male   
1     Surya Mohammad Sami        None        None    1996-03-14    Male   
2  Satyawan Hira Deshmukh        None        None    1997-03-14  Female   
3  Satyawan Hira Deshmukh        None        None    1998-03-14  Female   
4  Satyawan Hira Deshmukh        None        None    1999-03-14  Female   

   CBDT_INCOME_RANGE  TRAN_AMNT_CREDIT  TRAN_AMNT_DEBIT  PEER_GROUP_AMNT  \
0                nan  2264360596.00000  566453535.00000 2264360596.00000   
1                nan   566453534.00000              nan  566453534.00000   
2                nan   565000000.00000  566453535.00000  565000000.00000   
3                nan               nan  566453536.00000  566453536.00000   
4                nan               nan  566453537.00000  566453537.00000   

       PINCODE LATITUDE LONGITUDE OCCUPATION_BUSINESS ACCOUNT_COUNT  \
0 411026.00000     None      None                None          None   
1 411062.00000     None      None                None          None   
2 411062.00000     None      None                None          None   
3 411062.00000     None      None                None          None   
4 411062.00000     None      None                None          None   

  ADDRESS_COUNT  MOBILE_COUNT  TELEPHONE_COUNT  EMAIL_COUNT WATCHLIST_COUNT  \
0          None       1.00000              nan      1.00000            None   
1          None           nan              nan          nan            None   
2          None           nan              nan          nan            None   
3          None           nan              nan          nan            None   
4          None       1.00000              nan          nan            None   

  LEA_REQUESTS_COUNT LNK_REPORT_COUNT  STR_REPORT_COUNT  STR_AMOUNT_DEBIT  \
0               None             None           4.00000               nan   
1               None             None           1.00000               nan   
2               None             None           1.00000               nan   
3               None             None           1.00000               nan   
4               None             None           1.00000               nan   

   STR_AMOUNT_CREDIT  CTR_REPORT_COUNT  CTR_AMOUNT_DEBIT  CTR_AMOUNT_CREDIT  \
0                nan               nan               nan                nan   
1                nan               nan               nan                nan   
2                nan               nan               nan                nan   
3                nan               nan               nan                nan   
4                nan               nan               nan                nan   

   NTR_REPORT_COUNT  NTR_AMOUNT_DEBIT  NTR_AMOUNT_CREDIT PTR_REPORT_COUNT  \
0               nan               nan                nan             None   
1               nan               nan                nan             None   
2               nan               nan                nan             None   
3               nan               nan                nan             None   
4               nan               nan                nan             None   

  PTR_AMOUNT_DEBIT PTR_AMOUNT_CREDIT  CBWTR_REPORT_COUNT CBWTR_AMOUNT_DEBIT  \
0             None              None                 nan               None   
1             None              None                 nan               None   
2             None              None                 nan               None   
3             None              None 

In [26]:
temp = pd.merge(auto_high_df,summary_stg_df , on = 'PERSON_MASTER_ID',how = 'inner')

In [28]:
temp

PERSON_MASTER_ID       RULE_NO RISK_RULE_LIST  RISK_SCORE   PEER_GROUP_ID  \
0                51  [P001003000]     P001003000          10  Not Declared_0   
1               147  [P001003000]     P001003000          10  Not Declared_0   

          PAN   DIN CKYC_NUMBER               FULL_NAME FATHER_NAME  \
0  ABOPI0954B  None        None        Jrithi G Kahinas       Garan   
1  OMPPH5842J  None        None  Mahadev Kesari Shankar        None   

  MOTHER_NAME DATE_OF_BIRTH  GENDER  CBDT_INCOME_RANGE  TRAN_AMNT_CREDIT  \
0        None    1991-04-11    Male                nan               nan   
1        None    1993-04-11  Female                nan               nan   

   TRAN_AMNT_DEBIT  PEER_GROUP_AMNT      PINCODE LATITUDE LONGITUDE  \
0              nan          0.00000 560103.00000     None      None   
1    8550000.00000    8550000.00000 411041.00000     None      None   

  OCCUPATION_BUSINESS ACCOUNT_COUNT ADDRESS_COUNT  MOBILE_COUNT  \
0                None          None          None       1.00000   
1                None          None          None       1.00000   

   TELEPHONE_COUNT  EMAIL_COUNT WATCHLIST_COUNT LEA_REQUESTS_COUNT  \
0              nan      1.00000            None               None   
1              nan      1.00000            None               None   

  LNK_REPORT_COUNT  STR_REPORT_COUNT  STR_AMOUNT_DEBIT  STR_AMOUNT_CREDIT  \
0             None           1.00000               nan                nan   
1             None           1.00000     8550000.00000                nan   

   CTR_REPORT_COUNT  CTR_AMOUNT_DEBIT  CTR_AMOUNT_CREDIT  NTR_REPORT_COUNT  \
0               nan               nan                nan               nan   
1               nan               nan                nan               nan   

   NTR_AMOUNT_DEBIT  NTR_AMOUNT_CREDIT PTR_REPORT_COUNT PTR_AMOUNT_DEBIT  \
0               nan                nan             None             None   
1               nan                nan             None             None   

  PTR_AMOUNT_CREDIT  CBWTR_REPORT_COUNT CBWTR_AMOUNT_DEBIT  \
0              None                 nan               None   
1              None                 nan               None   

  CBWTR_AMOUNT_CREDIT CCR_REPORT_COUNT CCR_AMOUNT_DEBIT CCR_AMOUNT_CREDIT  \
0                None             None             None              None   
1                None             None             None              None   

  CREATE_DATE UPDATE_DATE  
0        None        None  
1        None        None

In [1]:
# Adding the flaged person in auto high rules to risk person table

for ind,row in temp.iterrows():
    conn.execute('''INSERT INTO [FINCORE_DB].[Fincore_Analytics].[RISK_PERSON] (PERSON_MASTER_ID, PEER_GROUP_ID, RISK_SCORE)
    values (?,?,?)''',(row['PERSON_MASTER_ID'],row['PEER_GROUP_ID'],10))

conn.commit()


NameError: name 'temp' is not defined

In [30]:
et_risk_df_per_rules = et_risk_df_per_rules[~et_risk_df_per_rules['PERSON_MASTER_ID'].isin(auto_high_df['PERSON_MASTER_ID'].to_list())]

In [31]:
et_risk_df_per_rules.head()

PERSON_MASTER_ID     RULE_NO RULE_VALUE  RISK_SCORE_STATIC  \
0                    1  P002001001          4                nan   
1                    1  P002001002          1                nan   
2                    1  P002001004          4                nan   
3                    1  P002001007          4                nan   
4                    1  P002001008          1                nan   
...                ...         ...        ...                ...   
1540               535  P002001002          1                nan   
1541               535  P002001004          2                nan   
1542               535  P002001007          1                nan   
1543               535  P002001009          1                nan   
1544               535  P002001015          1                nan   

                      CREATE_DATE  
0     2023-04-21 16:35:55.9700000  
1     2023-04-21 16:35:56.6800000  
2     2023-04-21 16:35:56.9966667  
3     2023-04-21 16:35:57.0533333  
4     2023-04-21 16:35:57.0766667  
...                           ...  
1540  2023-04-21 16:35:56.6800000  
1541  2023-04-21 16:35:56.9966667  
1542  2023-04-21 16:35:57.0533333  
1543  2023-04-21 16:35:57.1033333  
1544  2023-04-21 16:35:56.5100000  

[1530 rows x 5 columns]

In [32]:
et_risk_df_per_rules.drop_duplicates(inplace=True)

/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [43]:
error = pd.read_sql_query('''select PERSON_MASTER_ID,
a.rule_no,
COUNT(distinct rule_value)
from Fincore_Analytics.RISK_PERSON_SUMMARY_STG_RULES a with (nolock)
inner join Fincore_Analytics.RISK_lkp_rules b with (nolock) on a.rule_no = b.rule_no
where b.risk_score_static_flag = 0
group by PERSON_MASTER_ID,a.rule_no
having COUNT(distinct rule_value) > 1''',conn)
error

PERSON_MASTER_ID     rule_no   
0                39  P002001004  2
1               105  P002001004  2
2               534  P002003005  2

In [36]:
conn.close()

In [37]:
#changing vertical format to horizontal
pivot_per_df = et_risk_df_per_rules
pivot_per_df = pivot_per_df.pivot(index='PERSON_MASTER_ID', columns='RULE_NO', values=['RULE_VALUE'])
pivot_per_df.columns = [col[1] for col in pivot_per_df.columns.values]
pivot_per_df = pivot_per_df.reset_index()
cols = pivot_per_df.columns.to_list()
cols.append('CREATE_DATE')
pivot_per_df = pivot_per_df.merge(et_risk_df_per_rules,on = 'PERSON_MASTER_ID')[cols]

ValueError: Index contains duplicate entries, cannot reshape

In [31]:
pivot_per_df

PERSON_MASTER_ID     RULE_NO RULE_VALUE  RISK_SCORE_STATIC  \
0                    1  P002001001          4                nan   
1                    1  P002001002          1                nan   
2                    1  P002001004          4                nan   
3                    1  P002001007          4                nan   
4                    1  P002001008          1                nan   
...                ...         ...        ...                ...   
1540               535  P002001002          1                nan   
1541               535  P002001004          2                nan   
1542               535  P002001007          1                nan   
1543               535  P002001009          1                nan   
1544               535  P002001015          1                nan   

                      CREATE_DATE  
0     2023-04-21 16:15:42.8133333  
1     2023-04-21 16:15:43.4100000  
2     2023-04-21 16:15:43.7000000  
3     2023-04-21 16:15:43.7500000  
4     2023-04-21 16:15:43.8000000  
...                           ...  
1540  2023-04-21 16:15:43.4100000  
1541  2023-04-21 16:15:43.7000000  
1542  2023-04-21 16:15:43.7500000  
1543  2023-04-21 16:15:43.8200000  
1544  2023-04-21 16:15:43.3033333  

[1530 rows x 5 columns]

In [ ]:
data_ind_U = data_ind_U.merge(pivot_per_df,how = 'left',on = 'PERSON_MASTER_ID')
data_ind_U.head()

In [ ]:
data_ind_U = data_ind_U[~data_ind_U['PERSON_MASTER_ID'].isin(auto_high_df['PERSON_MASTER_ID'].to_list())]

In [ ]:
data_ind_U.head()

In [ ]:
data_ind_U.dtypes

In [ ]:
reqd_cols_per = list(set(non_stat_rules) - set(auto_high_rules)) #doing this to remove auto high rule from nan_stat_rule_list

# Threshold Calculations without Outliers

In [ ]:
from scipy import stats as s

In [ ]:
def compute_IQR(d,data):
    print("compute_IQR")
    q3 = np.quantile(data, 0.75)
    q1 = np.quantile(data, 0.25)
    iqr = ((q3-q1)* 1.5)
    outliers_left = q1 - iqr
    outliers_right = q3 + iqr
    print("outliers_right",outliers_right)
#     data = list(filter(lambda x: outliers_left <= x <= outliers_right, data))
    data = list(filter(lambda x: x <= outliers_right,d))

    return data,outliers_right

In [ ]:
def normal_outliers_removal(d):
    print("normal_outliers_removal")
    mean = np.mean(d)
    std = np.std(d)
    outliers_right = (mean + (3*std))
    data = list(filter(lambda x: x <= outliers_right, d))
    return data,outliers_right

In [ ]:
def thresh_cal(d,outliers_right):
    print("thresh")
    median = np.median(d)
    mean = np.mean(d)
    std = np.std(d)
    mode = int(s.mode(d)[0])

    
    if std >= 1:
        one = round(median+std)
        two = round(median+2*std)
        three = round(median+3*std)
        four = round(median+4*std)
        five = round(median+5*std)
        six = round(median+6*std)
        seven = round(median+7*std)
        eight = round(median+8*std)
        nine =round(median+9*std)
        ten = round(median+10*std)

    elif std > 0:
        one = np.ceil(median+std)
        two = np.ceil(one+std)
        three = np.ceil(two+std)
        four = np.ceil(three+std)
        five = np.ceil(four+std)
        six = np.ceil(five+std)
        seven = np.ceil(six+std)
        eight = np.ceil(seven+std)
        nine =np.ceil(eight+std)
        ten = np.ceil(nine+std)
        
        
    elif std == 0:
        one = np.ceil(median+std+1)
        two = np.ceil(one+std+1)
        three = np.ceil(two+std+1)
        four = np.ceil(three+std+1)
        five = np.ceil(four+std+1)
        six = np.ceil(five+std+1)
        seven = np.ceil(six+std+1)
        eight = np.ceil(seven+std+1)
        nine =np.ceil(eight+std+1)
        ten = np.ceil(nine+std+1)

        

    return one,two,three,four,five,six,seven,eight,nine,ten,mean,median,mode,std,outliers_right

In [ ]:
def not_normal_outliers_removal(d):
    print("not_normal_outliers_removal")
    data = [*set(d)]
    return compute_IQR(d,data)

In [ ]:


def compute_outlier(data,col):
    try:
#         print("data = ",data)
        d = data[col].dropna().astype('float64')
        d = d.astype('int64')
#         print("d = ",d)
    
        if (len(d) == 0):
            print("Not applicable")
            return None,None,None,None,None,None,None,None,None,None
          

        elif ( len(d) < 5):
            print("Not applicable")
            return thresh_cal(d,"NA") 
            
        
        
        
        else:
            
            if ( len(d) <= 150):
                stats,p_val = shapiro(d)
                if p_val > 0.05:
                    d,outliers_right = normal_outliers_removal(d)
                    return thresh_cal(d,outliers_right)
                else:
                    d,outliers_right = not_normal_outliers_removal(d)
                    return thresh_cal(d,outliers_right)   
                    
            else:
                stats,p_val  = kstest(d, 'norm')
                if p_val > 0.05:
                    d,outliers_right = normal_outliers_removal(d)
                    return thresh_cal(d,outliers_right)
                    
                else:
                    d,outliers_right = not_normal_outliers_removal(d)
                    return thresh_cal(d,outliers_right)
    
            
    except KeyError:
        return( None,None,None,None,None,None,None,None,None,None) 
    
    return (None,None,None,None,None,None,None,None,None,None)

In [ ]:
class Thresholds():
    def __init__(self,df,reqd_cols):
        self.data = df
        self.reqd_cols_U = reqd_cols
    def get_thresholds(self):
        # Python3 dict to contain threshold calculations-
        # Python3 lists to contain threshold values for desired columns-       
        self.class_dict = {}
        for col in self.reqd_cols_U:
            self.class_dict[col] = []
        
        self.class_dict['OCCUPATION_BUSINESS_CLEAN'] = []
        self.class_dict['Cluster_labels'] = []

        for occ in self.data['OCCUPATION_BUSINESS_CLEAN'].unique():
#             if occ == 'Others':
                for clabel in self.data.loc[self.data['OCCUPATION_BUSINESS_CLEAN'] == occ, 'Cluster_labels'].unique():

                    for col in self.reqd_cols_U:
                        print(f"Profession: {occ}, Cluster = {clabel},"
                        f" col = {col}; outlier"
                        f" = {compute_outlier(self.data[(self.data['OCCUPATION_BUSINESS_CLEAN'] == occ) & (self.data['Cluster_labels'] == clabel)], col)}"
                        )
                        # profession.append(occ)
                        # profession.append(clabel)
                        self.class_dict[col].append(compute_outlier(self.data[(self.data['OCCUPATION_BUSINESS_CLEAN'] == occ) & (self.data['Cluster_labels'] == clabel)], col))

                    self.class_dict['OCCUPATION_BUSINESS_CLEAN'].append(occ)
                    self.class_dict['Cluster_labels'].append(clabel)
#                 print(self.class_dict)
#                 break
#         self.class_dict['Type_of_profession'] = []
#         self.class_dict['Cluster_labels'] = []
                        
#         for occ in self.data['Type_of_profession'].unique():
#             for clabel in self.data.loc[self.data['Type_of_profession'] == occ, 'Cluster_labels'].unique():
#                 # print(f"{occ}, {clabel}")
#                 self.class_dict['Type_of_profession'].append(occ)
#                 self.class_dict['Cluster_labels'].append(clabel)
        
        self.threshold_data = pd.DataFrame(self.class_dict)
        return self.threshold_data
#         return self.class_dict

In [ ]:
data_ind_U

In [ ]:
data_ind_U.drop_duplicates(inplace = True)

In [ ]:
ind_U = Thresholds(data_ind_U,reqd_cols_per)
ind_data_thresholds_U = ind_U.get_thresholds()

In [ ]:
ind_data_thresholds_U

In [ ]:
config = ConfigParser()
config.read(r'/notebooks/notebooks/config.ini')
connection_string = config.get('SERVER', 'connection')
conn = pyodbc.connect(connection_string)
cursor = conn.cursor()

In [ ]:
weight_sql = pd.read_sql_query("SELECT * FROM [Fincore_DB].[Fincore_Analytics].[RISK_LKP_RULES]", conn)

In [ ]:
weight_table_ind_U = ind_data_thresholds_U.copy()

In [ ]:
for col in ind_data_thresholds_U.columns:
    if (col == 'OCCUPATION_BUSINESS_CLEAN') or (col == 'Cluster_labels') or (col == 'TRAN_AMNT'):
        pass
    else:
        weight_table_ind_U[col] = weight_sql[weight_sql['RULE_NO'] == col]['WEIGHTAGE'].values[0]

In [ ]:
config = ConfigParser()
config.read(r'/notebooks/notebooks/config.ini')
connection_string = config.get('SERVER', 'connection')
conn = pyodbc.connect(connection_string)
cursor = conn.cursor()

In [ ]:
rule_df = pd.read_sql_query("SELECT * FROM [FINCORE_DB].[Fincore_Analytics].[RISK_LKP_RULES] with (nolock) where IS_ACTIVE_FLAG = 1 and RISK_SCORE_STATIC_FLAG = 0 and RISK_TYPE <> 'Auto High Rules' and PERSON_TYPE = 'Individual'", conn)
reqd_cols_per = rule_df['RULE_NO'].to_list()
len(reqd_cols_per)

In [ ]:
col_with_data = data_ind_U.columns.to_list()

In [ ]:
col_with_data = [col for col in col_with_data if 'P0' in col]

In [ ]:
col_without_data = set(reqd_cols_per) - set(col_with_data)

In [ ]:
for col in col_without_data:
    data_ind_U[col] = np.nan

In [ ]:
ind_data_thresholds_U.to_csv(r"/data/threshold_individual_U.csv",index=False)
data_ind_U.to_csv(r"/data/Synthetic_data_Clustered_individual_U.csv",index=False)
weight_table_ind_U.to_csv(r"/data/weight_individual_U.csv",index=False)

In [1]:
print("Threshold computation V2 completed.")

Threshold computation V2 completed.
